In [ ]:
import re
from tqdm.auto import tqdm
import argilla as rg
import pandas as pd
import spacy
import sys
import pickle
from transformers import AutoTokenizer

In [ ]:
rg.init(
        api_url='http://procyon.tce.pi.gov.br:6902',
        api_key='owner.apikey',
        workspace='victor_silva'
    )

### ner_pt

In [ ]:
dataset = pd.read_csv('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/EntidadesNomeadas/ner_pt/ner_pt.csv')
dataset

In [ ]:
nlp = spacy.load("pt_core_news_lg")

In [ ]:
records = []
for index in tqdm(dataset.index, desc="Rotulando os dados", colour='yellow'):
    #entities = [(ent.label_, ent.start_char, ent.end_char) for ent in doc.ents]
    # Crie o registro TokenClassificationRecord
    print(eval(dataset['ner_tags'][index]))
    record = rg.TokenClassificationRecord(
        id=index,
        text=' '.join(eval(dataset.loc[index, 'tokens'])),
        tokens=eval(dataset.loc[index, 'tokens']),
        annotation = eval(dataset['ner_tags'][index]),
        status='Validated',
    )
    records.append(record)

In [ ]:
entidades = ['TIPO', 'NUMERO_PROCESSO', 'NOME']
settings = rg.TokenClassificationSettings(entidades)
rg.configure_dataset("diariosalepi", settings, workspace="victor_silva")
rg.log(records, "diariosalepi", workspace="victor_silva")

### Importando e transformando em DataFrame

#### Importando as duas bases para juntar no treinamento

In [ ]:
datasetrg = rg.load("datasetnerdiarios", workspace="victor_silva", query="status:Validated")
datasetrg1 = rg.load("diariosalepi", workspace="victor_silva", query="status:Validated")

In [ ]:
datasetCompleto = pd.concat([datasetrg.to_pandas(), datasetrg1.to_pandas()])
datasetCompleto.reset_index(drop=True, inplace=True)
datasetCompleto

In [ ]:
records = []
for index in tqdm(datasetCompleto.index, desc="Rotulando os dados", colour='yellow'):
    records.append(
        rg.TokenClassificationRecord(
            id = index,
            text=datasetCompleto['text'][index],
            tokens=datasetCompleto['tokens'][index],
            annotation = datasetCompleto['annotation'][index],
            status='Validated'
        )
    )

In [ ]:
entidades = ['TIPO', 'NUMERO_PROCESSO', 'NOME']
settings = rg.TokenClassificationSettings(entidades)
rg.configure_dataset("datasetnerdiariosfinal", settings, workspace="victor_silva")
rg.log(records, "datasetnerdiariosfinal", workspace="victor_silva")

In [ ]:
datasetrg = rg.load("datasetnerdiariosfinal", workspace="victor_silva", query="status:Validated")
dataset = datasetrg.prepare_for_training(framework="transformers", train_size=0.8)
dataset

In [ ]:
dataset.save_to_disk("/var/projetos/Jupyterhubstorage/victor.silva/HelBERT/Datasets/NamedEntities/dataNerDiarios")

#### Importando somente uma

In [ ]:
from datasets import load_from_disk

In [ ]:
dados = load_from_disk("../../../Datasets/NamedEntities/dataNerCrossFold")

In [ ]:
dados

In [ ]:
datasetrg = rg.load("datanertreino", workspace="victor_silva", query="status:Validated")
dataset = datasetrg.prepare_for_training(framework="transformers", train_size=1.0)

In [ ]:
mapping = {0: 'O', 1: 'B-LOCAL', 2: 'I-LOCAL', 3: 'B-OBJETO', 4: 'I-OBJETO'}

In [ ]:
import pandas as pd

In [ ]:
dados.to_pandas()

In [ ]:
datasetCompleto = dataset.to_pandas()
datasetCompleto

In [ ]:
for index in tqdm(datasetCompleto.index, desc="Rotulando os dados", colour='yellow'):
    datasetCompleto['ner_tags'][index] = [mapping[tag] for tag in datasetCompleto['ner_tags'][index]]
datasetCompleto

In [ ]:
for index in tqdm(datasetCompleto.index, desc="Rotulando os dados", colour='yellow'):
    datasetCompleto['tokens'][index] = " ".join(datasetCompleto['tokens'][index])

In [ ]:
datasetCompleto

In [ ]:
datasetCompleto.drop(columns=['id'], inplace=True)
datasetCompleto

In [ ]:
datasetCompleto.rename(columns={'tokens': 'text', 'ner_tags': 'labels'}, inplace=True)
datasetCompleto

In [ ]:
datasetCompleto.to_csv("../../../Datasets/NamedEntities/dataNerCrossFold.csv", index=False)

In [ ]:
dataset.save_to_disk("../../../Datasets/NamedEntities/dataNerCrossFold")

### NER Indicios

In [ ]:
dados = pd.read_csv('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Indicios/dataset_bid_notices_weak_sup_habilitacao.csv')
dados

In [ ]:
com_indicios = dados[(dados['certidao_protesto'] == 1) | (dados['certificado_boas_praticas'] == 1) | (dados['comprovante_localizacao'] == 1) | (dados['idoneidade_financeira'] == 1) | (dados['integralizado'] == 1) | (dados['licenca_ambiental'] == 1) | (dados['n_min_max_limitacao_atestados'] == 1)]
com_indicios

In [ ]:
com_indicios.drop_duplicates(subset=['text'], inplace=True)
com_indicios.dropna(subset=['text'], inplace=True)
com_indicios.reset_index(drop=True, inplace=True)
com_indicios

In [ ]:
print(com_indicios['certidao_protesto'].value_counts())
print(com_indicios['certificado_boas_praticas'].value_counts())
print(com_indicios['comprovante_localizacao'].value_counts())
print(com_indicios['idoneidade_financeira'].value_counts())
print(com_indicios['integralizado'].value_counts())
print(com_indicios['licenca_ambiental'].value_counts())
print(com_indicios['n_min_max_limitacao_atestados'].value_counts())

In [ ]:
dados_certidao_protesto = com_indicios[com_indicios['certidao_protesto'] == 1]
dados_certificado_boas_praticas = com_indicios[com_indicios['certificado_boas_praticas'] == 1]
dados_comprovante_localizacao = com_indicios[com_indicios['comprovante_localizacao'] == 1]
dados_idoneidade_financeira = com_indicios[com_indicios['idoneidade_financeira'] == 1]
dados_integralizado = com_indicios[com_indicios['integralizado'] == 1]
dados_licenca_ambiental = com_indicios[com_indicios['licenca_ambiental'] == 1]
dados_n_min_max_limitacao_atestados = com_indicios[com_indicios['n_min_max_limitacao_atestados'] == 1]

In [ ]:
dados_certidao_protesto = dados_certidao_protesto.sample(n=363).reset_index(drop=True)
dados_certificado_boas_praticas = dados_certificado_boas_praticas.sample(n=363).reset_index(drop=True)
dados_comprovante_localizacao = dados_comprovante_localizacao.sample(n=363).reset_index(drop=True)
dados_idoneidade_financeira = dados_idoneidade_financeira.sample(n=363).reset_index(drop=True)
dados_integralizado = dados_integralizado.sample(n=363).reset_index(drop=True)
dados_licenca_ambiental = dados_licenca_ambiental.sample(n=363).reset_index(drop=True)
dados_n_min_max_limitacao_atestados = dados_n_min_max_limitacao_atestados.sample(n=363).reset_index(drop=True)

In [ ]:
dados = pd.concat([dados_certidao_protesto, dados_certificado_boas_praticas, dados_comprovante_localizacao, dados_idoneidade_financeira, dados_integralizado, dados_licenca_ambiental, dados_n_min_max_limitacao_atestados])
dados

In [ ]:
dados.reset_index(drop=True, inplace=True)
dados

In [ ]:
def executaLimpeza(dataframe: pd.DataFrame, column: str, cased: bool, accents: bool) -> pd.DataFrame:
    """
    Executa a limpeza do DataFrame usando as classes Cleaner e Corretor.
    """
    sentencesDrop = []
    for indice in tqdm(dataframe.index, desc="Executando a Limpeza", colour='yellow'):
        texto = dataframe[column][indice]
        if isinstance(texto, str):
            texto = Cleaner().clear(texto)
            texto = Corretor(cased, accents).corrige_termos(texto)
            dataframe.at[indice, column] = texto
        else:
            sentencesDrop.append(indice)
    dataframe = dataframe.drop(sentencesDrop)
    return dataframe

In [ ]:
dados = executaLimpeza(dados, 'text', True, True)

In [ ]:
dados.dropna(subset=['text'], inplace=True)
dados.drop_duplicates(subset=['text'], inplace=True)
dados.reset_index(drop=True, inplace=True)
dados

In [ ]:
dados = dados.sample(frac=1).reset_index(drop=True)
dados

In [ ]:
dados.to_csv('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Indicios/indiciosNER.csv')

In [ ]:
dados = pd.read_csv('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Indicios/indiciosNER.csv')
dados.drop(columns=['Unnamed: 0'], inplace=True)
dados

In [ ]:
dados.to_csv('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Indicios/DatasetsNER/indiciosNERCompleto.csv', index=False)

### Usando o dataset com chunks

In [ ]:
import expressoes as exp
import re
from tqdm.auto import tqdm
import spacy
import pickle

In [ ]:
nlp = spacy.load("pt_core_news_lg")

In [ ]:
dados = pickle.load(open('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Indicios/DatasetsNER/indiciosNERCompleto_chunk.pkl', 'rb'))
dados

In [ ]:
def getEntities(expression: re, doc: str, label: int) -> list:
    ner_tags = []
    for match in re.finditer(expression, doc.text):
        span = doc.char_span(match.start(), match.end())
        if span:
            ner_tags.append((label, match.start(), match.end()))
    return ner_tags

In [ ]:
records = []
for index in tqdm(dados.index, desc="Rotulando os dados", colour='yellow'):
    entities = []
    text = dados['text'][index]
    doc = nlp(text)
    tokens = [token.text for token in doc]
    #tokens = text.split()
    certidao_protesto = getEntities(exp.certidao_protesto, doc, 'certidao_protesto')
    integralizado = getEntities(exp.integralizado, doc, 'integralizado')
    idoneidade_financeira = getEntities(exp.idoneidade_financeira, doc, 'idoneidade_financeira')
    comprovante_localizacao = getEntities(exp.comprovante_localizacao, doc, 'comprovante_localizacao')
    n_min_max_limitacao_atestados = getEntities(exp.n_min_max_limitacao_atestados, doc, 'n_min_max_limitacao_atestados')
    certificado_boas_praticas = getEntities(exp.certificado_boas_praticas, doc, 'certificado_boas_praticas')
    licenca_ambiental = getEntities(exp.licenca_ambiental, doc, 'licenca_ambiental')
    
    entities = certidao_protesto + integralizado + idoneidade_financeira + comprovante_localizacao + n_min_max_limitacao_atestados + certificado_boas_praticas + licenca_ambiental

    record = rg.TokenClassificationRecord(
        id=index,
        text=text,
        tokens = tokens,
        annotation=entities,
        status='Validated'
    )
    records.append(record)

In [ ]:
rg.log(records, "datasetnerindicios", workspace="victor_silva")

#### Importando

In [ ]:
datasetrg = rg.load("datasetnerindicios", workspace="victor_silva", query="status:Validated")
datasetrg

In [ ]:
dados = datasetrg.to_pandas()

In [ ]:
dataset = datasetrg.prepare_for_training(framework="transformers")

In [ ]:
datasetrg2 = datasetrg.select(
    (
        i for i in range(len(dataset)) 
        if i not in set(2787)
    )
)